In [13]:
import numpy as np

In [14]:
def generate_simple_edge_stream():
    edges = [(0,1,1,1), (1,2,2,2), (1,2,3,2), (0,2,3,2)]
    edges = sorted(edges, key=lambda x: x[2])
    return edges

In [15]:
def remove_dominated(tuples_list):
    dominated = set()
    for tup1 in tuples_list:
        for tup2 in tuples_list:
            if tup1 != tup2:
                if tup2[0] > tup1[0] and tup2[1]<=tup1[1]:
                    dominated.add(tup1)
                elif tup2[0]==tup1[0] and tup2[1]<tup1[1]:
                    dominated.add(tup1)
    
    # Remove dominated element from input list.
    cleared_list = [tup for tup in tuples_list if not tup in dominated]
    return cleared_list

In [16]:
tuple_list = [(1,5), (1,4), (3,7), (4,6)]
cleared_list = remove_dominated(tuple_list)
print("Before pruning:", tuple_list)
print("After pruning:", cleared_list)

Before pruning: [(1, 5), (1, 4), (3, 7), (4, 6)]
After pruning: [(1, 4), (4, 6)]


In [17]:
def fastest_path(network: list, source, t_alpha, t_omega):
    """Computes fastest paths from source to any other node in given network.

    Args:
        network (list): Network represented as edge stream.
        source (_type_): ID of source node.
        t_alpha (_type_): Lower bound of considered time interval.
        t_omega (_type_): Upper bound of considered time interval.
    """
    # Extract complete set of nodes from edge stream representation.
    node_set = set()
    for edge in network:
        node_set.add(edge[0])
        node_set.add(edge[1])
    
    # Init dict to store lists L_v for each node v.
    start_arrival_times = {node : set() for node in node_set}
    durations = {node : np.infty for node in node_set}
    durations[source] = 0
    
    # Process all edges in edge stream.
    for edge in network:
        u = edge[0]
        v = edge[1]
        t = edge[2]
        lam = edge[3]
        # Check if edge contained in time interval.
        if t < t_alpha or t + lam > t_omega:
            continue
        
        # Check if time point outside time interval.
        if t >= t_omega:
            break
         
        # Source and arrival time of source node is equal.
        if u == source:
            start_arrival_times[source].add((t,t))
            
        # Extract element in L_u with highest arrival time (and due to Lemma 11 also highest
        # starting time) and thus best potential to contribute to final fastest path.
        max_start_arrival = max(start_arrival_times[u], key=lambda x: x[1])
        s_v = max_start_arrival[0]
        a_v = t + lam
        
        # Update or insert start-arrival tuple for node v.
        v_updated = {x for x in start_arrival_times[v] if x[0]!=s_v}
        v_updated.add((s_v, a_v))
        start_arrival_times[v] = v_updated
        
        # Remove dominated elements.
        cleared_list = remove_dominated(start_arrival_times[v])
        start_arrival_times[v] = cleared_list
        
        # Check if to update fastes path distance of v.
        if a_v - s_v < durations[v]:
            durations[v] = a_v - s_v
        
        
    return durations

network = generate_simple_edge_stream()
print("Generated network:", network)
fastes_durations = fastest_path(network, 0, 0, 10)
print("Shortest durations:", fastes_durations)

Generated network: [(0, 1, 1, 1), (1, 2, 2, 2), (1, 2, 3, 2), (0, 2, 3, 2)]
Shortest durations: {0: 0, 1: 1, 2: 2}
